In [ ]:
!pip install pdfplumber langchain langchain_community langchain_core langchain_experimental faiss-cpu

In [40]:
from langchain_community.document_loaders import PDFPlumberLoader
from langchain_experimental.text_splitter import SemanticChunker
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.prompts import PromptTemplate
from langchain.chains.llm import LLMChain
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.chains import RetrievalQA

In [41]:
loader = PDFPlumberLoader("/content/Think python.pdf")
docs = loader.load()

In [59]:
text_splitter = SemanticChunker(HuggingFaceEmbeddings())
documents = text_splitter.split_documents(docs)

<ipython-input-59-b819d188c7c3>:1: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  text_splitter = SemanticChunker(HuggingFaceEmbeddings())


In [44]:
embedder = HuggingFaceEmbeddings()

<ipython-input-44-55e52f88ce51>:1: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embedder = HuggingFaceEmbeddings()


In [45]:
vector = FAISS.from_documents(documents, embedder)
vector.as_retriever(search_type = "similarity", search_kwargs = {"k":3})

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7f6439dcdb90>, search_kwargs={'k': 3})

In [ ]:
!pip install langchain-groq

In [47]:
from langchain_groq import ChatGroq


llm = ChatGroq(
    groq_api_key = "Your API-Key",
    temperature = 0,
    model_name = "deepseek-r1-distill-llama-70b"
)

In [48]:
llm.invoke("what is python")

AIMessage(content='<think>\n\n</think>\n\nPython is a high-level, interpreted programming language that is widely used for various purposes such as web development, scientific computing, data analysis, artificial intelligence, and more. Created in the late 1980s by Guido van Rossum, Python is known for its simplicity, readability, and ease of use, making it a great language for beginners and experts alike.\n\n### Key Features of Python:\n1. **Easy to Learn**: Python has a simple syntax with a focus on readability, often using English-like keywords and phrases.\n2. **Interpreted Language**: Python code is executed line by line, making it easier to write and test code quickly.\n3. **Versatile**: Python can be used for web development, scripting, data analysis, machine learning, automation, and more.\n4. **Cross-Platform**: Python can run on multiple operating systems, including Windows, macOS, and Linux.\n5. **Extensive Libraries**: Python has a vast collection of libraries and modules t

In [49]:
from langchain.prompts import PromptTemplate

In [50]:
# Define prompt

prompt = """
1. use the following pieces of context to answer the question at the end.
2. if you don't know the answer, just say that you don't know, don't try to make up an answer.
3. Keep the answer as concise as possible.
4. Always say "thanks for asking!" at the end of the answer.
5. If the question is not related to the context, politely respond that you are tuned to only answer questions that are related to the context.

Context :{context}
Question: {question}
Helpful Answer: """

QA_Chain_PROMPT = PromptTemplate.from_template(prompt)

In [51]:
llm_chain = LLMChain(
    llm = llm,
    prompt = QA_Chain_PROMPT,
    callbacks = None,
    verbose = True
)


In [52]:
document_prompt = PromptTemplate(
    input_variables = ["page_content", "source"],
    template = "Context: \ncontent: {page_content}\n source: {source}"
)


In [53]:
combine_documents_chain = StuffDocumentsChain(
    llm_chain = llm_chain,
    document_variable_name = "context",
    document_prompt = document_prompt,
    callbacks = None
)

In [54]:
from langchain.chains import RetrievalQA

In [55]:
from langchain.chains import RetrievalQA

In [60]:
# Create FAISS Vector Store
vector_store = FAISS.from_documents(documents, embedder)

# Define retriever
retriever = vector_store.as_retriever()


In [62]:
qa = RetrievalQA(combine_documents_chain=combine_documents_chain,
                 retriever=retriever,
                 return_source_documents=True,
                 verbose=True)



In [63]:
user_input = "what is an array?"
response = qa(user_input)["result"]

<ipython-input-63-bad7f01e388a>:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = qa(user_input)["result"]




> Entering new RetrievalQA chain...


> Entering new LLMChain chain...
Prompt after formatting:

1. use the following pieces of context to answer the question at the end.
2. if you don't know the answer, just say that you don't know, don't try to make up an answer.
3. Keep the answer as concise as possible.
4. Always say "thanks for asking!" at the end of the answer.
5. If the question is not related to the context, politely respond that you are tuned to only answer questions that are related to the context.

Context :Context: 
content: elements. There are several ways to create a new list; the simplest is to enclose the
elements in square brackets ( [ and ] ). For example, here is a list of two
integers. In [1]: numbers = [42, 123]
And here’s a list of three strings. In [2]: cheeses = ['Cheddar', 'Edam', 'Gouda']
The elements of a list don’t have to be the same type. The following list
contains a string, a float, an integer, and even another list. In [3]: t = ['spam', 2.0, 5, [10, 2

In [64]:
print(response)

<think>
Okay, so I need to figure out what an array is based on the given context. Let me go through each piece of context one by one.

First, the context talks about creating lists in Python using square brackets. It gives examples like numbers = [42, 123] and cheeses = ['Cheddar', 'Edam', 'Gouda']. It also mentions that lists can have mixed types, like t = ['spam', 2.0, 5, [10, 20]]. So, a list in Python is an ordered collection of elements that can be of any data type, including other lists.

Another context explains that lists are mutable, meaning they can be changed after creation. It shows how to access elements using index brackets, like cheeses[0] gives 'Cheddar'. It also demonstrates how to modify elements, such as numbers[1] = 17, which changes the list from [42, 123] to [42, 17]. This mutability is a key feature of lists.

The third context discusses variables and assignment. It explains that variables are names assigned to values using the equals operator. For example, n = 